# <center> Matching of the SDG </center>


As said before, to classify each project of investment of the Ecuadorian Government, we design a matching process between each project as one (or none) of the UN SDG. This required:

- To get the name of each project of investment of the Ecuadorian Government (See the PAI database).
- To get the specific SDG (SSDG) of each SDG